In [1]:

#useful: https://www.kdnuggets.com/2018/04/robust-word2vec-models-gensim.html

import csv
import nltk
import re
import numpy as np
import datetime
import time
import gensim, logging
import gensim
import gensim.corpora as corpora
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial import cKDTree
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
from scipy import sparse
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend([u'like',u'ebay', u'message', u'seller', u'sell', u'see', u'website', u'site', u'new', u'star', u'customer', u'wish', u'paypal', u'ca', u'page', u'android', u'shop', u'way', u'ad', u'let', u'add', u'team', u'able',u'library', u'hear', u'thing', u'really', u'continue', u'job', u'everyday', u'basic', u'choice', u'lil', u'month', u'save', u'come', u'ever', u'guy', u'keep', u'wifi', u'sometimes', u'artist', u'selection', u'choose', u'always', u'nothing', u'tablet', u'range', u'know', u'follow', u'daily', u'overall', u'give', u'totally', u'life', u'anything', u'everything', u'never', u'spotify', u'think', u'also', u'many', u'stuff', u'second', u'could', u'try', u'still', u'must', u'need', u'away', u'day', u'bit', u'number', u'make',u'year', u'lot', u'time',u'buy', u'song', u'even', u'minute', u'phone', u'gemitin', u'basta', u'say', u'everyone', u'service', u'take', u'bottom', u'panget', u'otherwise', u'yet', u'put', u'much', u'far', u'thank',  u'-PRON-', u'go', u'play',u'app', u'from', u'subject', u're', u'edu', u'use',  u'album', u'get', u'would', u'already',u'xe',u'vez', u'include', u'litt', u'sort', u'dayu', u'nice', u'dont', u'i', u'want', u'that'])
import warnings
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yubozhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('./android/viber/total_info.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

ratings = []
reviews = []
titles = []
dates = []
versions = []

for line in d:
    vals = line[0].split("******")
    ratings.append(float(vals[0]))
    reviews.append(vals[1])
    dates.append(vals[2])
    versions.append(vals[3])

In [3]:
reviews_new = []
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0
counter5 = 0
for i in range(len(ratings)):
    if ratings[i] == 1.0 and counter1<3947:
        counter1 = counter1 + 1
        reviews_new.append(reviews[i])
    if ratings[i] == 2.0 and counter2<3947:
        counter2 = counter2 + 1
        reviews_new.append(reviews[i])
    if ratings[i] == 3.0 and counter3<3947:
        counter3 = counter3 + 1
        reviews_new.append(reviews[i])
    if ratings[i] == 4.0 and counter4<3947:
        counter4 = counter4 + 1
        reviews_new.append(reviews[i])
    if ratings[i] == 5.0 and counter5<3947:
        counter5 = counter5 + 1
        reviews_new.append(reviews[i])



In [4]:
[13004,4338,3947, 4069, 10125]


[13004, 4338, 3947, 4069, 10125]

In [5]:
reviews=reviews_new

In [6]:
def remove_stopwords(texts, stop_words):
    corp=[]
    for doc in texts:
        fix_doc=[]
        for word in doc:
            if word not in stop_words:
                fix_doc.append(word)
        corp.append(fix_doc)
    return corp
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        if sent != []:
            doc = nlp(" ".join(sent)) 

            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
 
    return texts_out
def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str([sentence]), deacc=True)) 

In [7]:
data_words = list(sent_to_words(reviews))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)





/Users/yubozhang/anaconda/lib/python2.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_words_nostops = remove_stopwords(data_lemmatized, stop_words)
data_words_bigrams = make_bigrams(data_words_nostops)

data_lemmatized = data_words_bigrams

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


warnings.filterwarnings("ignore",category=DeprecationWarning)

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [11]:
#pyLDAvis.save_html(vis, 'lda1.html')
    

In [12]:
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Visualize the topics
#score =1
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/yubozhang/anaconda/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      23.509121        1       1 -0.072759  0.230172
4      21.826462        1       2 -0.208666 -0.290213
3      21.179821        1       3  0.346422 -0.108726
2      19.059944        1       4 -0.042143  0.118645
0      14.424649        1       5 -0.022854  0.050121, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
4714  Default  3271.000000          call  3271.000000  30.0000  30.0000
8227  Default  2658.000000         viber  2658.000000  29.0000  29.0000
8110  Default  2066.000000        update  2066.000000  28.0000  28.0000
8008  Default  1877.000000          good  1877.000000  27.0000  27.0000
2428  Default  1478.000000          work  1478.000000  26.0000  26.0000
1763  Default  1459.000000          send  1459.000000  25.0000  25.0000
153   Default  1383.000000           fix  1383.000000  24.0000  24.0000
1791  Default  1420.000000         video  1420.000000  23.0000  23.0000
2731  Default  1083.000000       sticker  1083.000000  22.0000  22.0000
4255  Default   783.000000       quality   783.000000  21.0000  21.0000
4016  Default   911.000000   application   911.000000  20.0000  20.0000
1501  Default   900.000000          last   900.000000  19.0000  19.0000
2302  Default   848.000000       problem   848.000000  18.0000  18.0000
1746  Default   817.000000          code   817.000000  17.0000  17.0000
5093  Default   761.000000    background   761.000000  16.0000  16.0000
2969  Default   734.000000        change   734.000000  15.0000  15.0000
2766  Default   669.000000       contact   669.000000  14.0000  14.0000
4951  Default   657.000000           bad   657.000000  13.0000  13.0000
1716  Default   739.000000          chat   739.000000  12.0000  12.0000
2774  Default   532.000000         photo   532.000000  11.0000  11.0000
795   Default   663.000000          open   663.000000  10.0000  10.0000
7036  Default   593.000000         voice   593.000000   9.0000   9.0000
4090  Default   645.000000      download   645.000000   8.0000   8.0000
1531  Default   481.000000         great   481.000000   7.0000   7.0000
1505  Default   553.000000    connection   553.000000   6.0000   6.0000
8249  Default   471.000000       version   471.000000   5.0000   5.0000
4742  Default   568.000000          quot   568.000000   4.0000   4.0000
3030  Default   522.000000         write   522.000000   3.0000   3.0000
5625  Default   465.000000         sound   465.000000   2.0000   2.0000
1509  Default   488.000000          load   488.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
4255   Topic5   782.652893       quality   783.332825   1.9354  -3.0028
2774   Topic5   532.062134         photo   532.733337   1.9350  -3.3887
1531   Topic5   480.618744         great   481.295624   1.9348  -3.4904
8249   Topic5   470.751678       version   471.423340   1.9348  -3.5111
5994   Topic5   306.498199       connect   307.172882   1.9340  -3.9402
8431   Topic5   253.377441        option   254.048004   1.9336  -4.1306
5175   Topic5   233.520554        delete   234.187805   1.9334  -4.2122
7314   Topic5   219.218491     reinstall   219.895264   1.9331  -4.2754
6584   Topic5   212.996597      activate   213.670639   1.9331  -4.3042
3449   Topic5   204.853195          poor   205.526871   1.9329  -4.3432
4130   Topic5   202.561020        camera   203.233490   1.9329  -4.3544
4507   Topic5   200.107956          lose   200.780289   1.9329  -4.3666
3659   Topic5   199.355850           ask   200.028259   1.9329  -4.3704
7110   Topic5   184.523376  conversation   185.194092   1.9326  -4.4477
3787   Topic5   183.723480         close   184.407608   1.9325  -4.4520
253    Topic5   169.197632        person   169.871506   1.9323  -4.5344
8348   Topic5   166.602997     developer   167.2737

In [13]:
pyLDAvis.save_html(vis, 'lda.html')

ImportError: No module named lda2vec